# Preparation

In [ ]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
import langchain as lc

import warnings
warnings.filterwarnings('ignore')

load_dotenv(find_dotenv())
openai.api_key = os.environ["OPENAI_API_KEY"]

In [ ]:
# Loading our vectorDB

import pickle
with open("vectorstore.pkl", "rb") as f:
    vectorstore = pickle.load(f)

# Agents

Agents use an LLM to determine which actions to perform and in what order. An action can be either using a tool and observing its output or returning it to the user. To use an agent, in addition to the concept of an LLM, it is important to understand a new concept and that of a "tool".

### 1.1 Tools
Tools are functions that agents can use to interact with the world. These tools can be common utilities (e.g. search), other chains, or even other agents.

In [ ]:
llm = lc.llms.OpenAI(temperature=0.5)

tool_names = ['llm-math'] # Let use math tool
tools = lc.agents.load_tools(tool_names, llm=llm)
tools

In [ ]:
# You can see LangChain.Agent create prompt for us. we don't need
# to create prompt again and again for same task

tool_list = [lc.agents.Tool(name='Math Tool',
             func = tools[0].run, # using above tools first index
             description="Tool to calculate, nothing else")
             ]

In [ ]:
agent = lc.agents.initialize_agent(tool_list,
                                   llm,
                                   agent=lc.agents.AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                                   verbose=True)
agent.run("How are you?")

In [ ]:
agent.run("What is 100 devided by 25?")

### 1.2 Custom Tools
You can also create your own tools by creating a class that inherits from BaseTool.

In [ ]:
from typing import Optional
from langchain.tools import BaseTool
from langchain.callbacks.manager import AsyncCallbackManagerForToolRun, CallbackManagerForToolRun

class CustomSearchTool(BaseTool):
    name = "restaurant search"
    description = "useful for when you need to answer questions about our restaurant"

    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        store = vectorstore.as_retriever()
        docs = store.get_relevant_documents(query)
        text_list = [doc.page_content for doc in docs]
        return "\n".join(text_list)
    
    async def _arun(self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")

In [ ]:
tools = [CustomSearchTool()]
agent = lc.agents.initialize_agent(tools, llm, agent=lc.agents.AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [ ]:
agent.run("When does the restaurant open?")